In [ ]:
%pip install -qU langchain langchain-community langchain-openai python-dotenv numexpr

In [5]:
import os
from dotenv import load_dotenv
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.utilities import BingSearchAPIWrapper


In [6]:
load_dotenv()

openai_api_key = os.environ["OPENAI_API_KEY"]
model_name = "gpt-3.5-turbo-0125"
llm = ChatOpenAI(
    model = model_name,
    openai_api_key = openai_api_key,
    temperature = 1,
    max_tokens = 1000
)

In [7]:
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiplies two integers together"""

    return first_int * second_int


@tool
def add(first_int: int, second_int: int) -> int:
    """Adds or Sums two integers together"""

    return first_int + second_int


@tool
def exponentize(base: int, exponent: int) -> int:
    """Exponentize the base to the exponent value"""

    return base ** exponent

@tool("BingSearch")
def search(query_string: str):
    """
    Useful to search for any kind of information and
    when you need to search the internet for any kinds of detailed information.
    """

    search = BingSearchAPIWrapper()
    return search.run(query_string)

In [ ]:
tools = [multiply, add, exponentize, search]
prompt = hub.pull("hwchase17/openai-tools-agent")
agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)

In [ ]:
question = "take 3 to the fifth power and multiply that by the sum of twelve and three, then square the whole result."

response = agent_executor.invoke({
    "input": question
})

print(response)